In [14]:
import numpy as np
import pandas as pd
import random

In [74]:
path_to_text = 'vectors.txt'
path_to_excel = '20200304_Aspects_Annotation.xlsx'
train_size = 400
annotated_size = 500
count_limit = 5

def get_final_dfs(
    path_to_text,
    path_to_excel,
    train_size,
    annotated_size,
    count_limit
):
    print("Reading .txt and .xlsx...")
    vectors_df, keywords_df = get_parsed_dfs(path_to_text, path_to_excel, count_limit=5)
    aspect_categories = get_aspect_categories(keywords_df)
    aspect_categories_numbered = dict(zip(range(1, 12), aspect_categories))
    no_of_categories = len(aspect_categories)
    matrix = get_matrix(vectors_df)
    centroids = get_centroids(keywords_df, matrix, aspect_categories, train_size)
    print("\nCalculating distances and similarities...")
    manhattan_distances, euclidean_distances, cos_sim, corr = get_distances(
        centroids, 
        matrix, 
        no_of_categories
    )
    manhattan_df = parse_to_df(manhattan_distances, vectors_df, no_of_categories)
    euclidean_df = parse_to_df(euclidean_distances, vectors_df, no_of_categories)
    cos_sim_df = parse_to_df(cos_sim, vectors_df, no_of_categories)
    corr_df = parse_to_df(corr, vectors_df, no_of_categories)
    
    def get_closest_index(row):
        dictionary = dict(zip(row, list(range(1, no_of_categories+1))))
        return dictionary[min(row)]

    def get_furthest_index(row):
        dictionary = dict(zip(row, list(range(1, no_of_categories+1))))
        return dictionary[max(row)]
    
    for df in [manhattan_df, euclidean_df]:
        df['CLOSEST'] = df.iloc[:, 1:no_of_categories+1].apply(
            get_closest_index, 
            axis=1
        )
        df['PREDICTION'] = [aspect_categories_numbered[x] for x in df['CLOSEST']]

    for df in [cos_sim_df, corr_df]:
        df['CLOSEST'] = df.iloc[:, 1:no_of_categories+1].apply(
            get_furthest_index, 
            axis=1
        )
        df['PREDICTION'] = [aspect_categories_numbered[x] for x in df['CLOSEST']]
    
    test_indices = get_test_indices(keywords_df, annotated_size, aspect_categories)
    for df in [manhattan_df, euclidean_df, cos_sim_df, corr_df]:
        get_accuracy(df, keywords_df, test_indices)
        
    return manhattan_distances, euclidean_distances, cos_sim, corr

In [75]:
get_final_dfs(
    path_to_text,
    path_to_excel,
    train_size,
    annotated_size,
    count_limit
)

Reading .txt and .xlsx...
Activities: 14
Environment and Ambience: 24
Experience: 24
F&B: 19
Facilities: 6
Family/Children family/ Friends: 13
Price: 13
Service: 3
Shopping: 7
Sightseeing/ Place: 103
Transportation and Accessibility: 51

Centroids found.

Calculating distances and similarities...
Accuracy: 0.43548387096774194
Accuracy: 0.5161290322580645
Accuracy: 0.532258064516129
Accuracy: 0.3225806451612903


(array([[ 99.428299  ,  98.53062779, 101.93011804, ..., 104.17046514,
          95.94584806, 101.11645671],
        [112.31006829, 111.47379187, 110.52465696, ..., 110.35272357,
         107.4549956 , 111.1347128 ],
        [127.23106543, 117.35154612, 119.56580979, ..., 131.12888243,
         115.42459671, 119.59029398],
        ...,
        [ 46.99797986,  41.71392304,  37.53582204, ...,  56.115556  ,
          41.99152295,  40.66729643],
        [ 46.95517471,  42.54350446,  38.64664404, ...,  55.713542  ,
          42.68226146,  43.02745333],
        [ 48.28976971,  44.59756288,  39.29774437, ...,  59.44166729,
          44.48785826,  44.06154402]]),
 array([[6.94715529, 7.00892117, 7.20183371, ..., 7.53926982, 6.78436462,
         7.1271808 ],
        [8.45263651, 8.37220199, 8.32221661, ..., 8.39437269, 8.0810994 ,
         8.32961824],
        [8.93338398, 8.46891214, 8.53837177, ..., 9.37955128, 8.28189415,
         8.55895532],
        ...,
        [3.37224009, 3.02121007, 2.7

In [16]:
def get_vectors_df(path_to_text): 
    vectors_df = pd.read_csv(path_to_text, delim_whitespace=True, header=None)
    parsed_col = []
    for keyword in list(vectors_df[0]):
        keyword = str(keyword)
        keyword = keyword.replace('_', ' ')
        parsed_col.append(keyword)
    vectors_df[0] = parsed_col
    vectors_df.columns = ['TEXT'] + list(range(1, vectors_df.shape[1]))
    return vectors_df

In [17]:
def get_keywords_df(path_to_excel, count_limit): 
    keywords_df = pd.read_excel(path_to_excel)
    keywords_df = keywords_df[keywords_df['count'] >= count_limit]
    parsed_col_2 = []
    for keyword in list(keywords_df['TEXT']):
        keyword = str(keyword)
        keyword = keyword.replace('â€™', "'")
        keyword = keyword.replace('ã©', 'é')
        parsed_col_2.append(keyword)
    keywords_df['TEXT'] = parsed_col_2
    keywords_df.drop_duplicates('TEXT', inplace=True) # There are duplicates after the .replace() functions
    return keywords_df

In [18]:
def get_parsed_dfs(path_to_text, path_to_excel, count_limit=5):
    vectors_df = get_vectors_df(path_to_text)
    keywords_df = get_keywords_df(path_to_excel, count_limit=count_limit)
    joint_df = vectors_df.merge(keywords_df)
    vectors_df = joint_df.iloc[:, :vectors_df.shape[1]]
    keywords_df = joint_df.iloc[:, vectors_df.shape[1]:]
    return vectors_df, keywords_df

In [19]:
def get_matrix(vectors_df):
    return vectors_df.iloc[:, 1:].values

In [20]:
def get_centroid(matrix):
    return np.mean(matrix, axis=0)

In [21]:
def get_manhattan(a, b):
    return np.abs(a - b).sum()

def get_euclidean(a, b):
    return np.linalg.norm(a-b)

def get_cos_sim(a, b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

def get_corr(a, b):
    return np.correlate(a, b)[0]

In [22]:
def get_aspect_categories(keywords_df):
    aspect_categories = list(set(keywords_df.ASPECT_CATEGORY_NAME))
    aspect_categories = [x for x in aspect_categories if x is not np.nan]
    aspect_categories.sort()
    return aspect_categories

In [42]:
def get_centroids(keywords_df, matrix, aspect_categories, train_size=400):
    df_train = keywords_df.iloc[:train_size, :]
    no_of_categories = len(aspect_categories)
    centroids = np.zeros((no_of_categories, 300))
    for i in range(no_of_categories):
        indices = df_train[df_train.ASPECT_CATEGORY_NAME == aspect_categories[i]].index
        print('{}: {}'.format(aspect_categories[i], len(indices)))
        centroids[i] = get_centroid(matrix[indices])
    print('\nCentroids found.')
    return centroids

In [73]:
def get_distances(centroids, matrix, no_of_categories):  # Returns tuple of 4 matrices
    manhattan_distances = np.zeros((matrix.shape[0], no_of_categories))
    euclidean_distances = np.zeros((matrix.shape[0], no_of_categories))
    cos_sim = np.zeros((matrix.shape[0], no_of_categories))
    corr = np.zeros((matrix.shape[0], no_of_categories))

    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            manhattan_distances[i, j] = get_manhattan(matrix[i], centroids[j])

    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            euclidean_distances[i, j] = get_euclidean(matrix[i], centroids[j])

    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            cos_sim[i, j] = get_cos_sim(matrix[i], centroids[j])

    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            corr[i, j] = get_corr(matrix[i], centroids[j])
                
    return manhattan_distances, euclidean_distances, cos_sim, corr

In [47]:
def parse_to_df(matrix, vectors_df, no_of_categories):
    final_df = pd.DataFrame(data=matrix)
    final_df['KEYWORD'] = vectors_df.TEXT
    cols = ['KEYWORD'] + [i for i in range(no_of_categories)]
    colnames = ['KEYWORD'] + [i for i in range(1, no_of_categories+1)]
    final_df = final_df.loc[:, cols]
    final_df.columns = colnames
    return final_df

In [67]:
def get_test_indices(df, annotated_size, aspect_categories):
    temp_df = df.iloc[train_size:annotated_size, :]
    return temp_df[temp_df.ASPECT_CATEGORY_NAME.isin(aspect_categories)].index

In [70]:
def get_accuracy(model, keywords_df, test_indices):
    sum = (model.PREDICTION[test_indices] == keywords_df.ASPECT_CATEGORY_NAME[test_indices]).sum()
    print("Accuracy: {}".format(sum/len(test_indices)))